In [1]:
#%%
import itertools
import math

import optuna
import pandas as pd

from scipy.stats import gmean
from scipy.stats.mstats import gmean

from src.simulador_v02 import obtener_skills, prioridad_x_serie, optuna_simular
from src.gymnasium_utils import *  
from src.datos_utils import DatasetTTP
from src.viz_utils import *
from src.optuna_utils import calcular_optimo_max_min

########################################################################
#-------------Cargar parámetros desde datos históricos------------------
########################################################################

dataset  = DatasetTTP.desde_csv_atenciones("data/fonasa_monjitas.csv.gz")
un_dia   = dataset.un_dia("2023-05-15").sort_values(by='FH_Emi', inplace=False)
"""Una tabla de atenciones para un dia"""

skills   = obtener_skills(un_dia)

series   = sorted(list({val for sublist in skills.values() for val in sublist}))
SLAs     = [(0.6, 30), (0.34, 35), (0.7, 45)]
niveles_servicio_x_serie = {s:random.choice(SLAs) for s in series}

planificacion = {'0': [{'inicio': '08:40:11',
   'termino': '10:07:40',
   'propiedades': {'skills': [5, 10, 11, 12, 14],
    'configuracion_atencion': 'Rebalse'}}],
 '1': [{'inicio': '08:40:11',
   'termino': '10:07:40',
   'propiedades': {'skills': [5, 10, 17],
    'configuracion_atencion': 'Alternancia'}}],
 '2': [{'inicio': '08:40:11',
   'termino': '10:07:40',
   'propiedades': {'skills': [5, 10, 11],
    'configuracion_atencion': 'Alternancia'}}],
 '3': [{'inicio': '08:40:11',
   'termino': '10:07:40',
   'propiedades': {'skills': [10, 11, 17], 'configuracion_atencion': 'FIFO'}}],
 '4': [{'inicio': '08:40:11',
   'termino': '10:07:40',
   'propiedades': {'skills': [5, 10, 11],
    'configuracion_atencion': 'Alternancia'}}],
 '11': [{'inicio': '08:40:11',
   'termino': '10:07:40',
   'propiedades': {'skills': [5, 10, 11],
    'configuracion_atencion': 'Alternancia'}}],
 '12': [{'inicio': '08:40:11',
   'termino': '10:07:40',
   'propiedades': {'skills': [11, 12, 14, 17],
    'configuracion_atencion': 'Rebalse'}}],
 '33': [{'inicio': '11:36:03',
   'termino': '13:02:33',
   'propiedades': {'skills': [5, 10, 12, 17],
    'configuracion_atencion': 'Alternancia'}}],
 '34': [{'inicio': '11:36:03',
   'termino': '13:02:33',
   'propiedades': {'skills': [14, 17],
    'configuracion_atencion': 'Alternancia'}}],
 '35': [{'inicio': '11:36:03',
   'termino': '13:02:33',
   'propiedades': {'skills': [11, 12, 14, 17],
    'configuracion_atencion': 'Rebalse'}}],
 '49': [{'inicio': '13:02:56',
   'termino': '14:30:23',
   'propiedades': {'skills': [5, 11], 'configuracion_atencion': 'Rebalse'}}],
 '50': [{'inicio': '13:02:56',
   'termino': '14:30:23',
   'propiedades': {'skills': [5, 10, 11, 12],
    'configuracion_atencion': 'Rebalse'}}],
 '51': [{'inicio': '13:02:56',
   'termino': '14:30:23',
   'propiedades': {'skills': [5, 11, 14, 17],
    'configuracion_atencion': 'FIFO'}}]}
"""lies!"""

########################################################################
#--------Reconstruir planificación desde trials guardados en sqlite------------------
########################################################################

# recomendaciones_db   = optuna.storages.get_storage("sqlite:///alejandro_objs.db") # TODO: MSSQL server someday
# resumenes            = optuna.study.get_all_study_summaries(recomendaciones_db)
# nombres              = [s.study_name for s in resumenes if "tramo_" in s.study_name]

# Seccion que deberia extraer un oprimo. 
# Guarda todos los estudios en un diccionario, con todos los trials, 
# scores_studios = {} # >> { 'tramo_0' : {0 : 0.09123} } 
# for un_nombre in nombres:
#     un_estudio            = optuna.multi_objective.load_study(study_name=un_nombre, storage=recomendaciones_db)
#     trials_de_un_estudio  = un_estudio.get_trials(deepcopy=False) #or pareto trials??
#     scores_studios        = scores_studios | {f"{un_nombre}":
#         { trial.number: calcular_optimo_max_min(trial.values)
#             # trial.values es una lista de SLA, tiempos de espera, etc. 
#             # calcular_optimo_max_min hace la division entre SLA / largo_fila_espera
#             # Cada key es el numero del trial. Kinda { 1: 0.089... } 
#                 for
#                     trial in trials_de_un_estudio if trial.state == optuna.trial.TrialState.COMPLETE}
#                     }    
#  
# trials_optimos          = extract_max_value_keys(scores_studios) # Para cada tramo, extrae el maximo, 
# # Que es una tupla de forma {'tramo_0' : (514, 0.459)} (osea el tramo : { n_trial, valor_trial })
# 
# planificaciones_optimas = {}   
# for k,v in trials_optimos.items():
#     un_estudio               = optuna.multi_objective.load_study(study_name=k, storage=recomendaciones_db)
#     trials_de_un_estudio     = un_estudio.get_trials(deepcopy=False)
#     planificaciones_optimas  = planificaciones_optimas | {f"{k}":
#         trial.user_attrs.get('planificacion')#calcular_optimo(trial.values)
#                 for
#                     trial in trials_de_un_estudio if trial.number == v[0]
#                     # trial.number == v[0] es el numero del trial, que se usa para hacer las planificaciones optimas
#                     }   
# 
# # Unifica las planificaciones en una unica planificacion
# # queda en el formato de planificacion, que deberia ser validada 
# planificacion                =  plan_unico([plan for tramo,plan in planificaciones_optimas.items()]) 

# {'0': [{'inicio': '08:40:11',
#    'termino': '10:07:40',
#    'propiedades': {'skills': [5, 10, 11, 12, 14],
#     'configuracion_atencion': 'Rebalse'}}],
#  '1': [{'inicio': '08:40:11',
#    'termino': '10:07:40',
#    'propiedades': {'skills': [5, 10, 17],
#     'configuracion_atencion': 'Alternancia'}}],
#  '2': [{'inicio': '08:40:11',
#    'termino': '10:07:40',
#    'propiedades': {'skills': [5, 10, 11],
#     'configuracion_atencion': 'Alternancia'}}],
#  '3': [{'inicio': '08:40:11',
#    'termino': '10:07:40',
# # ... # VALIDAR ESTE OBJETO! 

prioridades                  =  prioridad_x_serie(niveles_servicio_x_serie, 2, 1) # Reemplazado por un input/historico desde la DB
registros_atenciones, l_fila =  optuna_simular(planificacion, niveles_servicio_x_serie, un_dia, prioridades) # 
# Demora ~20 segs

/DeepenData/.miniconda/envs/flux/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
niveles_servicio_x_serie

{5: (0.7, 45),
 10: (0.7, 45),
 11: (0.34, 35),
 12: (0.6, 30),
 14: (0.34, 35),
 17: (0.6, 30)}

In [4]:
registros_atenciones.rename({'espera' : 'T_Esp'}, axis='columns')
# FH_Emi, IdEsc, IdSerie, T_Esp (= espera), T_Ate (opcional, 5 min para todo), 


,FH_Emi,IdSerie,T_Esp,IdEsc,T_Ate
0,2023-05-15 08:40:11,12,0,7,96
1,2023-05-15 08:40:54,14,0,10,157
2,2023-05-15 08:41:07,14,0,12,458
3,2023-05-15 08:41:18,14,0,11,861
4,2023-05-15 08:41:27,14,0,10,381
...,...,...,...,...,...
654,2023-05-15 14:12:59,10,18,NaN,NaN
655,2023-05-15 14:13:23,10,17,NaN,NaN
656,2023-05-15 14:14:26,12,16,NaN,NaN
657,2023-05-15 14:18:32,10,12,NaN,NaN
